In [ ]:
import pandas as pd
from bs4 import BeautifulSoup 
import re
import urllib
import requests
import shutil
from selenium import webdriver
import os
import time
import copy
from tqdm import tqdm
from pykospacing import spacing
import pandas as pd
import copy
import string, random ## generate random str package
from collections import OrderedDict ## repetition removal package

In [ ]:
def hangul(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return str(result)

def find_categories():
    test = driver.find_elements_by_css_selector("body > div.lyr_category_lst1 > ul > li:nth-child(1) > ul > li > a")
    Categories = []
    for i in test:
        Categories.append(i.text.split("\n")[:2])
    Count_Categories = len(Categories)
    return Categories, Count_Categories

def find_post_id():
    post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
    r = re.compile('logNo=.*')
    string = r.findall(post.get_attribute('ng-href'))
    return re.sub("logNo=",'',"".join(string))

def random_id():
    passkey='' # an empty str key
    for x in range(10): # length of the random passkeys

        if random.choice([1,2]) == 1:
            passkey += passkey.join(random.choice(string.ascii_letters)) # upper & lower cased letter
        else:
            passkey += passkey.join(random.choice(string.digits)) # numbers
    return passkey

def Spacing_text(text_list):
    spacing_list = []
    for i in text_list:
        if len(i) < 197:
            spacing_list.append(spacing(i))
        else:
            iteration = int(len(i) / 197)
            mod = len(i) % 197
            start = 0
            end = 197
            check = 0
            while True:
                # 시행횟수 < 몫
                if check < iteration:
                    spacing_list.append(spacing(i[start:end]))
                    start+=197
                    end+=197
                    check +=1
                else:
                    # 마지막 횟수 + 나머지 더 slice 
                    spacing_list.append(spacing(i[iteration*197:(iteration*197)+mod]))
                    break
    return spacing_list

In [ ]:
df = pd.read_csv("Filter_naver_category_18_drop_duplicated_ID.csv",encoding='utf-8')

In [ ]:
number = int(input("숫자를 입력해주세요: "))

User_id = df['ID'].iloc[number]
Category = df['Child'].iloc[number]

mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId=" + User_id
print(df['Child'].iloc[number])

driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
driver.get(mobile_url)
driver.implicitly_wait(3)

# 존재하지 않는 게시물 클릭 (for pass)
driver.find_element_by_class_name("btn_area").click()
driver.implicitly_wait(3)

# Category
temp_variable = input('Category 게시판을 선택해주세요.')

# Post_id 10개 append
Post_id_list = []
for i in driver.find_elements_by_class_name("card_section"):
    Post_id_list.append(i.get_attribute('id')[3:])
    
if  Post_id_list == []:
    for i in driver.find_elements_by_class_name("list"):
        Post_id_list.append(i.get_attribute('id')[3:])
driver.close()

# User_Table_Crawling

In [ ]:
# user_csv 작성
csv_name = 'Naver_user_table.csv'
col_name = "\t".join(["User_id","Blog_name","Blog_nickname","Blog_info_text","Count_neighbors","Count_visitors","Categories","Count_categories","Total_post","Credibility","Source","Blog_mobile_profile_img","Blog_mobile_cover_img"])

if not os.path.exists(csv_name):
    with open(csv_name, 'w') as f:
        f.write(col_name+'\n')

In [ ]:
Category_list = ["와인·술","육아·결혼","등산·낚시·레저","드라마·방송","교육·학문","건강·의학","차·커피·디저트",\
"국내여행","어학·외국어","만화·애니","IT·컴퓨터","자동차","시사·인문·경제","패션·뷰티","공연·전시","스포츠","맛집","사진",'무작위']

In [ ]:
## Category 폴더 만들기
Base_path = os.getcwd()
Category_path = "C:\\Users\\AjouHCI\\Desktop\\Archive\\Naver"
os.chdir(Category_path)
    
for directory in Category_list:
    if not os.path.exists(directory):
        os.makedirs(directory)
os.chdir(Base_path)

In [ ]:
## Path 설정
Naver_folder = "C:\\Users\\AjouHCI\\Desktop\\Archive\\Naver\\" 
Base_path = 'C:\\workspace\\Credibility_Research'
move_path =  Naver_folder + Category

In [ ]:
# source credibility check

Post_id = Post_id_list[0]
Source = "Naver"
Credibility = 1
mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId="+ User_id

# get mobile_url
driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
driver.get(mobile_url)
driver.implicitly_wait(3)

# 존재하지 않는 게시물 클릭
driver.find_element_by_class_name("btn_area").click()
driver.implicitly_wait(3)

# Blog_name, Blog_nickname, Blog_mobile_profile_img, Blog_info_text
Blog_name = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > h2 > a > span').text
Blog_nickname =driver.find_element_by_class_name("user_name").text

try:
    post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
    Blog_mobile_profile_img_url = post.get_attribute('ng-href')  
except:
    post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
    Blog_mobile_profile_img_url = post.get_attribute('href')
try:
    Blog_info_text = driver.find_element_by_class_name("text").text

except:
    Blog_info_text = ""

# Count_neighbors
neighbors_string = re.sub(",","",driver.find_element_by_class_name("count_buddy").text)
Count_neighbors = int(re.findall('\d+', neighbors_string)[0])

# Count_visitors
visitor_stirng = driver.find_elements_by_class_name('count')[0].text
Count_visitors = re.sub(",","",visitor_stirng.split("전체")[1]).strip()

# download_img
Mobile_cover_img_url = driver.find_element_by_class_name("cover_img").get_attribute('bg-lazy-img')

# category 버튼 클릭
driver.find_element_by_css_selector("#rego_cover > div.cover_cont > div.btn_area > div > div:nth-child(2) > a > span.txt").click()
driver.implicitly_wait(3)

# Total_Post, Categories, Count_Categories, 

Categories, Count_categories =  find_categories()
Total_post = driver.find_element_by_class_name("num").text

driver.close()

os.chdir(move_path)

# make category_folder
folder_name = User_id
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# download img
Save_path = Naver_folder + Category + '\\' + folder_name
os.chdir(Save_path)
cover_img = "Cover_" + folder_name + ".jpg"
Blog_mobile_profile_img = Category + "\\" + cover_img

profile_img = "Profile_" + folder_name + ".jpg"
Blog_mobile_cover_img =  Category + "\\" + profile_img

urllib.request.urlretrieve(Mobile_cover_img_url,cover_img)
urllib.request.urlretrieve(Blog_mobile_profile_img_url,profile_img)
os.chdir(Base_path)

save_content_list = [User_id,Blog_name,Blog_nickname,Blog_info_text,Count_neighbors,Count_visitors,Categories,Count_categories,Total_post,Credibility,Source,Blog_mobile_profile_img,Blog_mobile_cover_img]
save_content = "\t".join(list(map(str,save_content_list)))

with open(csv_name, 'a', encoding='utf-8') as f:
    f.write(save_content+"\n")

In [ ]:
pd.read_csv('Naver_user_table.csv',sep='\t',encoding='utf-8').tail()

# Post & Tag & Img Table_Crwaling

In [ ]:
# post_csv 작성
post_csv_name = 'Naver_post_table.csv'
post_col_name = "\t".join(["Post_id","User_id","Category","Title","Date","Structure","Structure_tag","Text","Count_space_mistake","Map_exist","Media_count","Heart_count","Sticker_count"])
if not os.path.exists(post_csv_name):
    with open(post_csv_name, 'w') as f:
        f.write(post_col_name +'\n')

        
# img_csv 작성
img_csv_name = 'Naver_img_table.csv'
img_col_name = "\t".join(["Img_id","Post_id"])
if not os.path.exists(img_csv_name):
    with open(img_csv_name, 'w') as f:
        f.write(img_col_name +'\n')

# tag_csv 작성
tag_csv_name = 'Naver_tag_table.csv'
tag_col_name = "\t".join(["Post_id","Post_tag"])
if not os.path.exists(tag_csv_name):
    with open(tag_csv_name, 'w') as f:
        f.write(tag_col_name +'\n')

In [ ]:
User_id_list = [User_id] * 10
blog_list = list(zip(copy.copy(User_id_list),copy.copy(Post_id_list)))

for user_id, post_id in tqdm(blog_list):
    User_id = user_id
    Post_id = post_id
    Category = Category
    
    url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
    r = requests.get(url)
    bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
    if 'u_rmc_btn' in bs or  'ytp-button' in bs:
        mdeia_exist = 1
    else: 
        mdeia_exist = 0

    #title
    Title = bs.find("h3", {"class": "se_textarea"})
    if (Title == None):
        Title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
    if (Title != None):
        Title = Title.text.strip()
    else:
        Title = "TITLE ERROR"

    #date
    # Append_value
    Date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})
    if Date == None:
        Date = bs.find("p",{"class":"date fil5 pcol2 _postAddDate"})
        
    Date_text = re.sub("\n","",Date.text)
    Date = re.sub("\t","",Date_text)

    #Teg, Content_structure, Text

    # structure
    structure = bs.find("div", {"id": "postViewArea"})
    if structure == None:
        structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})
    structure_tag = structure.find_all(['p','img'])
    
    
    # only tag & texf extract
    tag_list = []
    text_list = []
    structure_list = []
    for tag in structure_tag:

        if "<img " in str(tag):
            tag_list.append('<img>')
            structure_list.append('<img>')
            
        temp_tag = re.sub('\xa0','',tag.text)
        temp_tag = re.sub('\u200b','',temp_tag)
        
        if len(temp_tag) > 1: 
            
            if "<p " in str(tag):
                tag_list.append('<p>')
                structure_list.append('<p>')
                check = 0

                if '<br/>' in str(tag):
                    tag_list.append('<br>')
                    structure_list.append('<br>')

                if "<span>" in str(tag):
                    for span in tag:
                        if "<span></span>" not in str(span) and "<span>" in str(span):
                            text_list.append(span.text)
                            structure_list.append(span.text)
                            tag_list.append('<span>')
                            structure_list.append('<span>')
                else:
                    text_list.append(tag.text)
                    structure_list.append(tag.text)
                    tag_list.append("</p>")
                    structure_list.append('</p>')
                    check = 1
                if check != 1:
                    tag_list.append('</p>')
                    structure_list.append('</p>')
        else:
            pass
    
    structure_list = list(map(lambda x : re.sub('\xa0','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\u200b','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\n','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\t','',x),structure_list))
    # Structure
    Structure = "|".join(structure_list)
    
    text_list = list(map(lambda x : re.sub("\xa0","",x),text_list))
    text_list = list(map(lambda x : re.sub("\u200b","",x),text_list))
    text_list = list(map(lambda x : re.sub("\n","",x),text_list))
    text_list = list(map(lambda x : re.sub("\t","",x),text_list))
    filter_text = list(filter(lambda x: len(x)>1 ,text_list))
    
    Text = "".join(list(map(lambda x : x.strip(),filter_text)))    
    Space_text = "".join(Spacing_text(filter_text))
    Count_space_mistake = len(Space_text)-len(Text)
    
    # only tag
    Structure_tag = "|".join(tag_list)

    # image download    
    imgs = structure.find_all('img')
    Save_path = Naver_folder + Category + '\\' + User_id
    
    Map_exist = 0
    Sticker_count = 0
    for img in imgs:
        
        if "스티커 이미지" in str(img):
            img_name = "Sticker_" + User_id + "_" + random_id() + ".jpg"
            Sticker_count += 1
        else:
            img_name =  User_id + "_" + random_id() + ".jpg"

        img_url = re.sub("\u200b","",str(img['src']))
        
        try:
            urllib.request.urlretrieve(img_url,img_name)
            shutil.move(img_name,Save_path)
        
        except UnicodeEncodeError:
            try:
                if 'map' not in str(img_url):
                    driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
                    driver.get(url)
                    driver.implicitly_wait(3)
                    img = driver.find_element_by_tag_name('img')
                    src = re.sub("\u200b","",str(img.get_attribute('src')))
                    urllib.request.urlretrieve(src, Save_path)
                    shutil.move(img_name,move_folder)
                    driver.close()
                else:
                    Map_exist = 1
            except:
                pass
        except:
            pass
 
        
        Img_id = "Naver" + '\\'+ Category + '\\' + User_id + '\\'+ img_name
        save_img_content = Img_id + "\t" + Post_id
        
        with open(img_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_img_content + "\n")
        post_img = Category + "\\" + User_id+ '\\'+ img_name
        
    # Post_tag, Media_count
    driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
    driver.get(url)
    driver.implicitly_wait(10)
    tags = driver.find_element_by_class_name('post_footer_contents').text.split("#")[1:]
    Heart_count = driver.find_element_by_class_name('postre').text
    if Heart_count == '' or Heart_count == 'NaN':
        for i in range(0,5):
            Heart_count = driver.find_element_by_class_name('postre').text
            tags = driver.find_element_by_class_name('post_footer_contents').text.split("#")[1:]
    Media_count = 0
    Media_check = driver.find_elements_by_tag_name('iframe')

    for Media in Media_check:
        if 'Player' in str(Media.get_attribute('src')):
            Media_count += 1

    for Post_tag in tags:
        save_tag = Post_id + "\t" + Post_tag
        with open(tag_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_tag + "\n")

    # make a save list
    save_post = "\t".join(list(map(str,[Post_id,User_id,Category,Title,Date,Structure,Structure_tag,Text,Count_space_mistake,Map_exist,Media_count,Heart_count,Sticker_count])))
    with open(post_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_post + "\n")
        
    driver.close()

In [ ]:
pd.read_csv(post_csv_name,sep='\t').tail()

In [ ]:
pd.read_csv(img_csv_name,sep='\t').tail()

In [ ]:
pd.read_csv(tag_csv_name,sep='\t').tail()

# Page_Post & Tag & Img Table_Crwaling

In [ ]:
pages = 11
check_structure = []

## 무작위 User_folder 만들기
Base_path = os.getcwd()
Category_path_random = Category_path + "\\무작위"
os.chdir(Category_path_random)


if not os.path.exists(User_id):
    os.makedirs(User_id)

os.chdir(Base_path)


for page in tqdm(range(1,pages)):

    Post_id = User_id + "_" + str(page)
    Category = '무작위'

    url = "http://blog.naver.com/PostList.nhn?blogId=" + User_id + "&currentPage=" + str(page)
    r = requests.get(url)
    bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
    try:
        if '아직 작성된 글이 없습니다.' in bs.find('div',{'class':"new_blog_inner2"}).text:
            break
    except:
        pass

    #title
    Title = bs.find("h3", {"class": "se_textarea"})
    if (Title == None):
        Title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
    if (Title != None):
        Title = Title.text.strip()
    else:
        Title = "TITLE ERROR"

    #date

    Date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})
    if Date == None:
        Date = bs.find("p",{"class":"date fil5 pcol2 _postAddDate"})

    Date_text = re.sub("\n","",Date.text)
    Date = re.sub("\t","",Date_text)

    #Teg, Content_structure, Text

    # structure
    structure = bs.find("div", {"id": "postViewArea"})
    if structure == None:
        structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})
    structure_tag = structure.find_all(['p','img'])

    if structure in check_structure:
        break
    else:
        check_structure.append(structure)
        
    # only tag & texf extract
    tag_list = []
    text_list = []
    structure_list = []
    for tag in structure_tag:

        if "<img " in str(tag):
            tag_list.append('<img>')
            structure_list.append('<img>')

        temp_tag = re.sub('\xa0','',tag.text)
        temp_tag = re.sub('\u200b','',temp_tag)

        if len(temp_tag) > 1: 

            if "<p " in str(tag):
                tag_list.append('<p>')
                structure_list.append('<p>')
                check = 0

                if '<br/>' in str(tag):
                    tag_list.append('<br>')
                    structure_list.append('<br>')

                if "<span>" in str(tag):
                    for span in tag:
                        if "<span></span>" not in str(span) and "<span>" in str(span):
                            text_list.append(span.text)
                            structure_list.append(span.text)
                            tag_list.append('<span>')
                            structure_list.append('<span>')
                else:
                    text_list.append(tag.text)
                    structure_list.append(tag.text)
                    tag_list.append("</p>")
                    structure_list.append('</p>')
                    check = 1
                if check != 1:
                    tag_list.append('</p>')
                    structure_list.append('</p>')
        else:
            pass

    structure_list = list(map(lambda x : re.sub('\xa0','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\u200b','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\n','',x),structure_list))
    structure_list = list(map(lambda x : re.sub('\t','',x),structure_list))
    # Structure
    Structure = "|".join(structure_list)

    text_list = list(map(lambda x : re.sub("\xa0","",x),text_list))
    text_list = list(map(lambda x : re.sub("\u200b","",x),text_list))
    text_list = list(map(lambda x : re.sub("\n","",x),text_list))
    text_list = list(map(lambda x : re.sub("\t","",x),text_list))
    filter_text = list(filter(lambda x: len(x)>1 ,text_list))

    Text = "".join(list(map(lambda x : x.strip(),filter_text)))    
    Space_text = "".join(Spacing_text(filter_text))
    Count_space_mistake = len(Space_text)-len(Text)

    # only tag
    Structure_tag = "|".join(tag_list)

    # image download    
    imgs = structure.find_all('img')
    Save_path = Naver_folder + Category + '\\' + User_id

    Map_exist = 0
    Sticker_count = 0
    for img in imgs:

        if "스티커 이미지" in str(img):
            img_name = "Sticker_" + User_id + "_" + random_id() + ".jpg"
            Sticker_count += 1
        else:
            img_name =  User_id + "_" + random_id() + ".jpg"

        img_url = re.sub("\u200b","",str(img['src']))

        try:
            urllib.request.urlretrieve(img_url,img_name)
            shutil.move(img_name,Save_path)

        except UnicodeEncodeError:
            try:
                if 'map' not in str(img_url):
                    driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
                    driver.get(url)
                    driver.implicitly_wait(3)
                    img = driver.find_element_by_tag_name('img')
                    src = re.sub("\u200b","",str(img.get_attribute('src')))
                    urllib.request.urlretrieve(src, Save_path)
                    shutil.move(img_name,move_folder)
                    driver.close()
                else:
                    Map_exist = 1
            except:
                pass 
        except:
            pass

        Img_id = "Naver" + '\\'+ Category + '\\' + User_id + '\\'+ img_name
        save_img_content = Img_id + "\t" + Post_id

        with open(img_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_img_content + "\n")
        post_img = Category + "\\" + User_id+ '\\'+ img_name

    # Post_tag, Media_count, Heart_count
    driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
    driver.get(url)
    driver.implicitly_wait(10)

    tags = driver.find_element_by_class_name('post_footer_contents').text.split("#")[1:]
    Heart_count = driver.find_element_by_class_name('postre').text

    if Heart_count == '' or Heart_count == 'NaN':
        for i in range(0,5):
            Heart_count = driver.find_element_by_class_name('postre').text
            tags = driver.find_element_by_class_name('post_footer_contents').text.split("#")[1:]

    Media_count = 0
    Media_check = driver.find_elements_by_tag_name('iframe')

    for Media in Media_check:
        if 'Player' in str(Media.get_attribute('src')):
            Media_count += 1

    for Post_tag in tags:
        save_tag = Post_id + "\t" + Post_tag
        with open(tag_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_tag + "\n")

    # make a save list
    save_post = "\t".join(list(map(str,[Post_id,User_id,Category,Title,Date,Structure,Structure_tag,Text,Count_space_mistake,Map_exist,Media_count,Heart_count,Sticker_count])))

    with open(post_csv_name, 'a', encoding='utf-8') as f:
            f.write(save_post + "\n")
    driver.close()

In [ ]:
pd.read_csv(post_csv_name,sep='\t').tail()

In [ ]:
pd.read_csv(img_csv_name,sep='\t').tail()

In [ ]:
pd.read_csv(tag_csv_name,sep='\t').tail()